In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.17.0


### Define the image size and batch size

In [3]:
IMG_SIZE = (224, 224)  
BATCH_SIZE = 32

#### Create ImageDataGenerators for training and validation

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

### Load and preprocess data

In [6]:
train_generator = train_datagen.flow_from_directory(
    'data',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 1803 images belonging to 7 classes.


In [7]:
test_generator = test_datagen.flow_from_directory(
    'data',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 1803 images belonging to 7 classes.


## Build the CNN Model

In [8]:
# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

#### Combine base model and custom layers

In [9]:
model = Model(inputs=base_model.input, outputs=predictions)

#### Freeze the base model layers

In [10]:
for layer in base_model.layers:
    layer.trainable = False

#### Compile the model

In [11]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

## Train the Model

In [12]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10


d:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


57/57 ━━━━━━━━━━━━━━━━━━━━ 906s 16s/step - accuracy: 0.4379 - loss: 5.6694 - val_accuracy: 0.8946 - val_loss: 0.3009
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 950s 17s/step - accuracy: 0.7733 - loss: 0.5852 - val_accuracy: 0.9207 - val_loss: 0.2195
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 903s 16s/step - accuracy: 0.8203 - loss: 0.4524 - val_accuracy: 0.9379 - val_loss: 0.1884
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 924s 16s/step - accuracy: 0.8267 - loss: 0.4534 - val_accuracy: 0.9329 - val_loss: 0.1857
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 885s 16s/step - accuracy: 0.8146 - loss: 0.4968 - val_accuracy: 0.9495 - val_loss: 0.1498
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 876s 15s/step - accuracy: 0.8552 - loss: 0.3812 - val_accuracy: 0.9445 - val_loss: 0.1518
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 866s 15s/step - accuracy: 0.8476 - loss: 0.3863 - val_accuracy: 0.9534 - val_loss: 0.1363
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 997s 18s/step - accuracy: 0.8637 - loss: 0.3834 - val_accuracy: 0.9479 - val_

## Evaluate the Model

#### Get predictions

In [13]:
test_steps_per_epoch = np.math.ceil(test_generator.samples / BATCH_SIZE)
y_pred = model.predict(test_generator, steps=test_steps_per_epoch)
y_pred_classes = np.argmax(y_pred, axis=1)

C:\Users\Rashid Iqbal\AppData\Local\Temp\ipykernel_8288\2205458674.py:1: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  test_steps_per_epoch = np.math.ceil(test_generator.samples / BATCH_SIZE)


57/57 ━━━━━━━━━━━━━━━━━━━━ 445s 8s/step


#### Get true labels

In [14]:
y_true = test_generator.classes

#### Calculate metrics

In [15]:
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

In [16]:
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9512
Precision: 0.9523
Recall: 0.9512
F1 Score: 0.9506


#### Unfreeze some layers of the base model

In [17]:
for layer in base_model.layers[15:]:
    layer.trainable = True

#### # Recompile the model

In [18]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

### Fine-tune the model

In [19]:
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 990s 17s/step - accuracy: 0.8902 - loss: 0.3373 - val_accuracy: 0.9684 - val_loss: 0.0874
Epoch 2/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1021s 18s/step - accuracy: 0.9050 - loss: 0.2799 - val_accuracy: 0.9684 - val_loss: 0.0839
Epoch 3/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1071s 19s/step - accuracy: 0.9215 - loss: 0.2280 - val_accuracy: 0.9750 - val_loss: 0.0699
Epoch 4/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 971s 17s/step - accuracy: 0.9332 - loss: 0.1991 - val_accuracy: 0.9756 - val_loss: 0.0642
Epoch 5/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1490s 26s/step - accuracy: 0.9318 - loss: 0.1931 - val_accuracy: 0.9773 - val_loss: 0.0624
Epoch 6/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1228s 22s/step - accuracy: 0.9503 - loss: 0.1535 - val_accuracy: 0.9823 - val_loss: 0.0491
Epoch 7/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 1045s 18s/step - accuracy: 0.9486 - loss: 0.1510 - val_accuracy: 0.9811 - val_loss: 0.0495
Epoch 8/10
57/57 ━━━━━━━━━━━━━━━━━━━━ 960s 17s/step - accuracy: 0.9562 - loss: 0.1434 - val_accurac

# Load and Preprocess the Image
### You need to load the image and preprocess it to match the input requirements of your model.

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

def load_and_preprocess_image(img_path, img_size=(224, 224)):
    # Load the image and resize it
    img = image.load_img(img_path, target_size=img_size)
    
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    
    # Expand dimensions to match the model's input shape
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess the image for the VGG16 model
    img_array = preprocess_input(img_array)
    
    return img_array

## Make Predictions
### Use the preprocessed image to make predictions with your model.

In [21]:
def predict_image_class(model, img_path):
    # Load and preprocess the image
    img_array = load_and_preprocess_image(img_path)
    
    # Predict the class
    predictions = model.predict(img_array)
    
    # Get the predicted class index
    predicted_class = np.argmax(predictions, axis=1)
    
    return predicted_class, predictions

## Interpret the Results
#### Convert the predicted class index into a readable label.

In [22]:
# Get the class labels from the training generator
class_labels = list(train_generator.class_indices.keys())

def interpret_prediction(predicted_class, class_labels):
    # Decode the prediction into a readable label
    predicted_label = class_labels[predicted_class[0]]
    return predicted_label

# Example Usage

### Combine the above functions to test your image.

In [24]:
# Path to your test image
img_path = 'test-image1.jpeg'

predicted_class, predictions = predict_image_class(model, img_path)

predicted_label = interpret_prediction(predicted_class, class_labels)

print(f'Predicted class: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 901ms/step
Predicted class: bike


# Save the trained model

In [25]:
model.save('path/to/your/saved/model.h5')